In [10]:
## Step 1: Import Libraries and Define Helper Functions

# Import necessary libraries
import pandas as pd
import numpy as np
from IPython.display import HTML
import matplotlib.colors as mcolors
import sys
import os

# Add project root to Python path
notebook_dir = os.path.abspath('')
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added project root to Python path: {project_root}")

# Import project modules
try:
    from src.data.data_fetcher import get_recent_games
    from src.utils.game_analyzer import analyze_game
    from src.utils.config import COLORS, METRICS
    print("Successfully imported project modules")
except ImportError as e:
    print(f"Error importing project modules: {e}")
    print("Make sure you're running this notebook from the 'notebooks' directory")
    raise

# Define colors for different grades (same as in dashboard)
grade_colors = {
    'A+': '#BB86FC',
    'A': '#9D65F9',
    'B+': '#03DAC6',
    'B': '#00B5A3',
    'C+': '#CF6679',
    'C': '#B04759',
    'D': '#FF7597',
    'N/A': '#666666'
}

# Function to format numeric values
def format_numeric(value):
    """Format numeric values to one decimal place"""
    return f"{value:.1f}" if isinstance(value, (int, float)) else value

# Function to create color-coded HTML table
def create_styled_table(df):
    """Create a stylish HTML table with grade color coding"""
    # Copy dataframe to avoid warnings
    styled_df = df.copy()
    
    # Format numeric columns
    numeric_columns = [
        'Period Scores', 'Extra Periods', 'Lead Changes', 'Buzzer Beater',
        'FG3_PCT', 'Star Performance', 'Margin', 'Total Score', 'Average Margin'
    ]
    
    for col in numeric_columns:
        if col in styled_df.columns:
            styled_df[col] = styled_df[col].apply(format_numeric)
    
    # Create CSS style for table
    styles = [
        dict(selector="th", props=[("background-color", "#1E1E1E"),
                                 ("color", "#BB86FC"),
                                 ("padding", "10px"),
                                 ("border", "1px solid #333333")]),
        dict(selector="td", props=[("padding", "10px"),
                                 ("border", "1px solid #333333")])
    ]
    
    # Apply Pandas styling
    return styled_df.style \
        .set_table_styles(styles) \
        .apply(lambda x: [f'background-color: {grade_colors.get(val, "#121212")}' 
                         if x.name == "Grade" else '' for val in x], axis=0) \
        .set_properties(**{
            'background-color': '#121212',
            'color': '#FFFFFF',
            'font-family': '"Consolas", "Monaco", "Courier New", monospace'
        })



Added project root to Python path: c:\Users\Emir.Isakovic\OneDrive - Fellowmind Sweden\Desktop\Nba API - Copy
Successfully imported project modules


In [15]:
## Step 2: Fetch and Analyze Data

# Fetch and analyze games
recent_games = get_recent_games(days=5, max_games=5)
filtered_recent_games = recent_games[recent_games['MATCHUP'].str.contains('@')]
unique_recent_games = filtered_recent_games.drop_duplicates(subset=['GAME_ID'])

if unique_recent_games.empty:
    print("No away games found, using all recent games instead.")
    unique_recent_games = recent_games.drop_duplicates(subset=['GAME_ID'])

# Analyze games
results = []
for _, game in unique_recent_games.iterrows():
    game_id = game['GAME_ID']
    matchup = game['MATCHUP']
    game_date = game['GAME_DATE'].strftime('%Y-%m-%d')
    
    print(f"Analyzing game: {matchup} on {game_date}")
    game_results = analyze_game(game_id, game_date, matchup)
    results.append(game_results)

# Create DataFrame
results_df = pd.DataFrame(results)

# Display formatted table
display(HTML("<h3>Detailed Games Overview</h3>"))
styled_table = create_styled_table(results_df)
display(styled_table)


## Step 3: Filter and Analyze Specific Games

# Example 1: Show only high-grade games (A or A+)
high_grade_games = results_df[results_df['Grade'].isin(['A+', 'A'])]
display(HTML("<h4>High Grade Games (A or A+)</h4>"))
display(create_styled_table(high_grade_games))

# Example 2: Show games with high Total Score
high_score_games = results_df[results_df['Total Score'] > results_df['Total Score'].mean()]
display(HTML("<h4>Games with Above Average Total Score</h4>"))
display(create_styled_table(high_score_games))

# Example 3: Show games with many Lead Changes
exciting_games = results_df[results_df['Lead Changes'] > results_df['Lead Changes'].median()]
display(HTML("<h4>Games with Many Lead Changes</h4>"))
display(create_styled_table(exciting_games))





Analyzing game: TOR @ ORL on 2025-03-02
Analyzing game: NYK @ MIA on 2025-03-02
Analyzing game: CHI @ IND on 2025-03-02
Analyzing game: POR @ CLE on 2025-03-02
Analyzing game: DEN @ BOS on 2025-03-02


,Game ID,Game Date,Teams,Period Scores,Extra Periods,Lead Changes,Buzzer Beater,FG3_PCT,Star Performance,Margin,Total Score,Grade,Average Margin
0,0022400870,2025-03-02,TOR @ ORL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N/A,0.0
1,0022400869,2025-03-02,NYK @ MIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N/A,0.0
2,0022400868,2025-03-02,CHI @ IND,50.0,0.0,5.0,0.0,5.0,nan,18.9,nan,D,7.4
3,0022400867,2025-03-02,POR @ CLE,42.2,5.0,2.1,0.0,5.0,8.0,25.0,87.3,A,0.0
4,0022400866,2025-03-02,DEN @ BOS,32.4,0.0,0.0,0.0,5.0,4.0,21.0,62.4,D,6.6


,Game ID,Game Date,Teams,Period Scores,Extra Periods,Lead Changes,Buzzer Beater,FG3_PCT,Star Performance,Margin,Total Score,Grade,Average Margin
3,0022400867,2025-03-02,POR @ CLE,42.2,5.0,2.1,0.0,5.0,8.0,25.0,87.3,A,0.0


,Game ID,Game Date,Teams,Period Scores,Extra Periods,Lead Changes,Buzzer Beater,FG3_PCT,Star Performance,Margin,Total Score,Grade,Average Margin
3,0022400867,2025-03-02,POR @ CLE,42.2,5.0,2.1,0.0,5.0,8.0,25.0,87.3,A,0.0
4,0022400866,2025-03-02,DEN @ BOS,32.4,0.0,0.0,0.0,5.0,4.0,21.0,62.4,D,6.6


,Game ID,Game Date,Teams,Period Scores,Extra Periods,Lead Changes,Buzzer Beater,FG3_PCT,Star Performance,Margin,Total Score,Grade,Average Margin
2,0022400868,2025-03-02,CHI @ IND,50.0,0.0,5.0,0.0,5.0,nan,18.9,nan,D,7.4
3,0022400867,2025-03-02,POR @ CLE,42.2,5.0,2.1,0.0,5.0,8.0,25.0,87.3,A,0.0


In [14]:
## Step 4: Create Your Own Filtering
# Template for custom filtering
# Copy and modify this code to create your own filters
custom_filter = results_df[
    # Example conditions (remove # and modify conditions)
    #(results_df['Grade'].isin(['A+', 'A', 'B+'])) &  # Select specific grades
    #(results_df['Total Score'] > 70) &  # Set Total Score threshold
    #(results_df['Lead Changes'] > 10)  # Set Lead Changes threshold
    True  # Remove this line when adding your own conditions
]

# Display filtered data
display(HTML("<h4>Your Custom Filter</h4>"))
display(create_styled_table(custom_filter))

,Game ID,Game Date,Teams,Period Scores,Extra Periods,Lead Changes,Buzzer Beater,FG3_PCT,Star Performance,Margin,Total Score,Grade,Average Margin
3,0022400867,2025-03-02,POR @ CLE,42.2,5.0,2.1,0.0,5.0,8.0,25.0,87.3,A,0.0
6,0022400864,2025-03-01,GSW @ PHI,46.8,0.0,0.0,0.0,5.0,10.0,19.3,81.1,B+,7.3
11,0022400859,2025-02-28,LAC @ LAL,50.0,0.0,0.0,0.0,4.4,7.3,25.0,86.7,A,4.9
12,0022400858,2025-02-28,MIN @ UTA,50.0,0.0,1.4,0.0,5.0,6.0,25.0,87.4,A,2.7
14,0022400856,2025-02-28,NYK @ MEM,50.0,0.0,0.0,0.0,4.6,3.3,25.0,82.9,B+,2.9
15,0022400855,2025-02-28,TOR @ CHI,44.2,5.0,0.0,0.0,5.0,2.7,25.0,81.8,B+,0.0
16,0022400854,2025-02-28,IND @ MIA,50.0,0.0,3.6,0.0,5.0,10.0,23.9,92.5,A,5.4


## Usage Tips

1. **Sorting**: Click column headers to sort data
2. **Copying**: Select cells to copy data
3. **Export**: Right-click and select 'Copy' to export data
4. **Filtering**: Use the template for custom filtering and adjust conditions

## Common Filtering Examples

- Find games with specific grades:
  ```python
  results_df[results_df['Grade'].isin(['A+', 'A'])]
  ```

- Find games with high scores:
  ```python
  results_df[results_df['Total Score'] > results_df['Total Score'].mean()]
  ```

- Find close games:
  ```python
  results_df[results_df['Average Margin'] < 5]
  ```

- Combine multiple conditions:
  ```python
  results_df[
      (results_df['Grade'].isin(['A+', 'A'])) &
      (results_df['Lead Changes'] > 10) &
      (results_df['Average Margin'] < 5)
  ]
  ``` 